# Rankability Predicting Sensitivity
## Lichess Tournaments Dataset

Goal of this notebook is to process the data using massey and colley for a bunch of different parameters. Save those results for later processing.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import copy
import os
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.stats import skew
from tqdm import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from datetime import datetime
import re

In [ ]:
from pathlib import Path
home = str(Path.home())
home

In [ ]:
import sys
sys.path.insert(0,"%s/rankability_toolbox_dev"%home)
import pyrankability

In [ ]:
sys.path.insert(0,"%s/sensitivity_study/src"%home)
from sensitivity_tests import *
from utilities import *
from base import *

In [ ]:
tournaments = pd.read_csv('https://raw.githubusercontent.com/IGARDS/sensitivity_study/master/experiments/T-lichess-Run.csv')
games={}
time_format = '%Y-%m-%d %H:%M:%S'
tournaments['date_time'] = tournaments['date_time'].apply(lambda x: datetime.strptime(re.split("[.+]", x)[0], time_format))
tournaments['min_time'] = tournaments['date_time'].groupby(tournaments['tournament_id']).transform('min')
tournaments['max_time'] = tournaments['date_time'].groupby(tournaments['tournament_id']).transform('max')
tournaments['halftime'] = tournaments['min_time'] + (tournaments['max_time'] - tournaments['min_time']) / 2
tournaments['before_halftime'] = tournaments['date_time'] < tournaments['halftime']
tournament_ids = tournaments.tournament_id.unique()
for tournament_id in tournament_ids:
    games[tournament_id] = tournaments[tournaments.tournament_id == tournament_id]
games[tournament_id]

In [ ]:
# Note to future self: Parameters from FODS paper but might need to be optimized
direct_thress = [0,1,2]
spread_thress = [0,3,6]
weight_indirects = [0.25,0.5,1.]
domains_ranges = [('all','madness'),('madness','madness')]

# fracs represent how much of the data to include
fracs = [0.5,0.6,0.7,0.8,0.9,1.]

In [ ]:
import itertools
import joblib

In [ ]:
massey_rankings = {}
colley_rankings = {}
massey_rs = {}
colley_rs = {}

outer_keys = list(itertools.product(domains_ranges,years))
for domain_range,year in tqdm(outer_keys):
    # set the team_domain
    team_domain = None
    if domain_range[0] == 'madness':
        team_domain = madness_teams[year]
    elif domain_range[0] == 'all':
        team_domain = all_teams[year]

    # set the team_range
    team_range = None
    if domain_range[1] == 'madness':
        team_range = madness_teams[year]
    elif domain_range[1] == 'all':
        team_range = all_teams[year]

    columns = ["frac","direct_thres","spread_thres","weight_indirect"]+team_range
    massey_rankings[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_rankings[(domain_range,year)] = pd.DataFrame(columns=columns)
    massey_rs[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_rs[(domain_range,year)] = pd.DataFrame(columns=columns)

    game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                            "team1_score":games[year]['points1'],
                            "team1_H_A_N": games[year]['H_A_N1'],
                            "team2_name":games[year]['team2_name'],
                            "team2_score":games[year]['points2'],
                            "team2_H_A_N": games[year]['H_A_N1'],
                            "date": games[year]['date']
                           }).sort_values(by='date').drop('date',axis=1)
    mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
    game_df = game_df.loc[mask]

    keys = list(itertools.product(fracs,direct_thress,spread_thress,weight_indirects))

    def compute(frac,direct_thres,spread_thres,weight_indirect):
        upper = int(len(game_df)*frac)
        game_df_sample = game_df.iloc[:upper,:]

        map_func = lambda linked: pyrankability.construct.colley_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
        colley_matrix,colley_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        colley_matrix = colley_matrix.reindex(index=team_range,columns=team_range)
        colley_b = colley_b.reindex(team_range)
        mask = colley_b.isna()
        colley_b = colley_b.loc[~mask]
        colley_matrix = colley_matrix.loc[~mask,~mask]
        inxs = list(np.where(mask)[0])
        ranking1,r1 = pyrankability.construct.ranking_from_matrices(colley_matrix.fillna(0),colley_b,inxs)

        map_func = lambda linked: pyrankability.construct.massey_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
        massey_matrix,massey_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        massey_matrix = massey_matrix.reindex(index=team_range,columns=team_range)
        massey_b = massey_b.reindex(team_range)
        mask = massey_b.isna()
        massey_b = massey_b.loc[~mask]
        massey_matrix = massey_matrix.loc[~mask,~mask]    
        inxs = list(np.where(mask)[0])    
        ranking2,r2 = pyrankability.construct.ranking_from_matrices(massey_matrix.fillna(0),massey_b,inxs)
        ranking_values1 = [frac,direct_thres,spread_thres,weight_indirect]+list(ranking1)
        ranking_values2 = [frac,direct_thres,spread_thres,weight_indirect]+list(ranking2)
        r_values1 = [frac,direct_thres,spread_thres,weight_indirect]+list(r1)
        r_values2 = [frac,direct_thres,spread_thres,weight_indirect]+list(r2)
        return pd.Series(ranking_values1,index=columns),pd.Series(ranking_values2,index=columns),pd.Series(r_values1,index=columns),pd.Series(r_values2,index=columns)

    #frac,direct_thres,spread_thres,weight_indirect = keys[0]
    #compute(frac,direct_thres,spread_thres,weight_indirect)
    results = Parallel(n_jobs=-1)(delayed(compute)(frac,direct_thres,spread_thres,weight_indirect) for frac,direct_thres,spread_thres,weight_indirect in keys)

    c = 0
    for i,key in enumerate(keys):
        frac,direct_thres,spread_thres,weight_indirect = key
        massey,colley,massey_r,colley_r = results[i]
        massey.name = c
        colley.name = c
        colley_r.name=c
        massey_r.name=c
        massey_rankings[(domain_range,year)] = massey_rankings[(domain_range,year)].append(massey)
        colley_rankings[(domain_range,year)] = colley_rankings[(domain_range,year)].append(colley)
        massey_rs[(domain_range,year)] = massey_rs[(domain_range,year)].append(massey_r)
        colley_rs[(domain_range,year)] = colley_rs[(domain_range,year)].append(colley_r)
        c+=1

In [ ]:
list(colley_rankings[(domain_range,year)].set_index('frac').loc[[0.5,1.0]].reset_index().groupby(['direct_thres','spread_thres','weight_indirect']))[0]

In [ ]:
feature_name = 'intersection_0.5_to_1.0'
def compute_score(data):
    k=10
    s = 0
    c=0
    for i1,i2 in itertools.combinations(range(len(data)),2):
        s+=len(set(data[i1][:k]).intersection(set(data[i2][:k])))/k
        c+=1
    return s/c

def results_to_frame(results,method,domain_range,year):
    t = results.to_frame()
    t.columns=[feature_name]
    t['Method'] = method
    t['Year']=year
    t['Domain']=domain_range[0]
    t['Range']=domain_range[1]
    t.reset_index(inplace=True)
    return t

ms = pd.DataFrame(columns=[feature_name,'Method','Year'])
for domain_range,year in tqdm(outer_keys):
    # set the team_range
    team_range = None
    if domain_range[1] == 'madness':
        team_range = madness_teams[year]
    elif domain_range[1] == 'all':
        team_range = all_teams[year]    
    grouped = colley_rankings[(domain_range,year)].set_index('frac').loc[[0.5,1.0]].reset_index().groupby(['direct_thres','spread_thres','weight_indirect'])
    results = grouped.apply(lambda df: compute_score(df[team_range].astype(int).values.tolist()))
    ms = ms.append(results_to_frame(results,'Colley',domain_range,year))
    grouped = massey_rankings[(domain_range,year)].set_index('frac').loc[[0.5,1.0]].reset_index().groupby(['direct_thres','spread_thres','weight_indirect'])
    results = grouped.apply(lambda df: compute_score(df[team_range].astype(int).values.tolist()))
    ms = ms.append(results_to_frame(results,'Massey',domain_range,year))
ms

In [ ]:
sensitivity_target = ms.set_index('Method').loc['Massey'].reset_index().set_index(['Year','direct_thres','spread_thres','weight_indirect','Domain','Range'])['intersection_0.5_to_1.0']

sensitivity_data = {}
for year in tqdm(years):
    sensitivity_data[year] = {}
    for frac in fracs:
        game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                                "team1_score":games[year]['points1'],
                                "team1_H_A_N": games[year]['H_A_N1'],
                                "team2_name":games[year]['team2_name'],
                                "team2_score":games[year]['points2'],
                                "team2_H_A_N": games[year]['H_A_N1'],
                                "date": games[year]['date']
                               }).sort_values(by='date').drop('date',axis=1)
        mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
        game_df = game_df.loc[mask]
        upper = int(len(game_df)*frac)
        game_df_sample = game_df.iloc[:upper,:]
        sensitivity_data[year][f"frac={frac}"]=game_df_sample
        
description = """
This dataset represents the sensitivity problem as defined as follows:

A practitioner wants to predict the degree to which a season of the NCAA Men’s Basketball 
will be likely to change after the second half of the season is played. This is in the context of a Massey
with a specific set of parameters

direct_thress = [0,1,2]
spread_thress = [0,3,6]
weight_indirects = [0.25,0.5,1.]
domains_ranges = [('all','madness'),('madness','madness')]
"""
joblib.dump({'description':description,'target':sensitivity_target,'data':sensitivity_data,'other':{'madness_teams':madness_teams,'remaining_games':remaining_games}},"/disk/rankability_datasets/sensitivity_study/problem_0002a.joblib.z")

In [ ]:
sensitivity_target = ms.set_index('Method').loc['Colley'].reset_index().set_index(['Year','direct_thres','spread_thres','weight_indirect','Domain','Range'])['intersection_0.5_to_1.0']

sensitivity_data = {}
for year in tqdm(years):
    sensitivity_data[year] = {}
    for frac in fracs:
        game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                                "team1_score":games[year]['points1'],
                                "team1_H_A_N": games[year]['H_A_N1'],
                                "team2_name":games[year]['team2_name'],
                                "team2_score":games[year]['points2'],
                                "team2_H_A_N": games[year]['H_A_N1'],
                                "date": games[year]['date']
                               }).sort_values(by='date').drop('date',axis=1)
        mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
        game_df = game_df.loc[mask]
        upper = int(len(game_df)*frac)
        game_df_sample = game_df.iloc[:upper,:]
        sensitivity_data[year][f"frac={frac}"]=game_df_sample
        
description = """
This dataset represents the sensitivity problem as defined as follows:

A practitioner wants to predict the degree to which a season of the NCAA Men’s Basketball 
will be likely to change after the second half of the season is played. This is in the context of a Colley
with a specific set of parameters

direct_thress = [0,1,2]
spread_thress = [0,3,6]
weight_indirects = [0.25,0.5,1.]
domains_ranges = [('all','madness'),('madness','madness')]
"""
joblib.dump({'description':description,'target':sensitivity_target,'data':sensitivity_data,'other':{'madness_teams':madness_teams,'remaining_games':remaining_games}},"/disk/rankability_datasets/sensitivity_study/problem_0002b.joblib.z")

In [ ]:
ms

In [ ]:
sensitivity_target = ms.set_index('Method').loc['Massey'].reset_index().set_index([''])

groupby('Year')['mean_top10_intersection'].mean()

sensitivity_data = {}
for year in tqdm(years):
    sensitivity_data[year] = {}
    for frac in fracs:
        game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                                "team1_score":games[year]['points1'],
                                "team1_H_A_N": games[year]['H_A_N1'],
                                "team2_name":games[year]['team2_name'],
                                "team2_score":games[year]['points2'],
                                "team2_H_A_N": games[year]['H_A_N1'],
                                "date": games[year]['date']
                               }).sort_values(by='date').drop('date',axis=1)
        mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
        game_df = game_df.loc[mask]
        upper = int(len(game_df)*frac)
        game_df_sample = game_df.iloc[:upper,:]
        sensitivity_data[year][f"frac={frac}"]=game_df_sample
        
description = """
This dataset represents the sensitivity problem as defined as follows:

A practitioner wants to predict the degree to which the second half of the season of the NCAA
Men’s Basketball will change after they play the games. The practioner believes that Massey is 
the best. They feel that a direct win has to be above 1 point difference and that the spread threshold
must be 3. They also feel that indirects are 0.25 weight of a direct game. They feel like indirects
between any teams are important.

direct_thress = [0,1,2]
spread_thress = [0,3,6]
weight_indirects = [0.25,0.5,1.]
domains_ranges = [('all','madness'),('madness','madness')]
"""
joblib.dump({'description':description,'target':sensitivity_target,'data':sensitivity_data,'other':{'madness_teams':madness_teams,'remaining_games':remaining_games}},"/disk/rankability_datasets/sensitivity_study/problem_0002b.joblib.z")
